Input: arquivo com a mensagem em claro + Chave RSA pública do destinatário + Algoritmo simétrico

In [22]:
from Crypto.PublicKey import RSA

In [23]:
def generateKeyRSA(secretCode:str, legthModuleRSA:int) -> bytes:
    key = RSA.generate(legthModuleRSA)
    encriptedKey = key.export_key(passphrase=secretCode, pkcs=8, protection="scryptAndAES128-CBC")
    return encriptedKey

In [24]:
def generateKeyPrivate(legthModuleRSA:int):
    key = RSA.generate(legthModuleRSA)
    encriptedKey = key.export_key()
    return encriptedKey

In [25]:
def generateKeyPublic(legthModuleRSA:int):
    key = RSA.generate(legthModuleRSA)
    encriptedKey = key.publickey().export_key()
    return encriptedKey

In [26]:
def writeKey(key:bytes, mode:int) -> bool:
    if mode == 0:
        with open("./input/rsaKey.bin", "wb") as archive:
            try:
                archive.write(key)
                return True
            except:
                return False
    elif mode == 1:
        with open("./input/private.pem", "wb") as archive:
            try:
                archive.write(key)
                return True
            except:
                return False
    elif mode == 2:
        with open("./input/receiver.pem", "wb") as archive:
            try:
                archive.write(key)
                return True
            except:
                return False

In [27]:
def main():
    rsa = generateKeyRSA("cachaça", 2048)
    private = generateKeyPrivate(2048)
    public = generateKeyPublic(2048)
    writeKey(rsa, 0)
    writeKey(private, 1)
    writeKey(public, 2)


if __name__ == "__main__":
    main()